# Test VGG16 Architcture without Augmentation

# Importing Libraries

In [ ]:
import os
from pathlib import Path

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install pandas

In [ ]:
!pip install matplotlib

In [ ]:
!pip install sklearn

In [ ]:
!pip install scikit-learn

In [ ]:
!pip install pillow

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import numpy as np

%matplotlib inline

# Designing a Convolution Neural Network (CNN): 

In [ ]:
# Imports
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions

VGG_types = {
    "VGG11": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "VGG13": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "VGG16": [
        64,
        64,
        "M",
        128,
        128,
        "M",
        256,
        256,
        256,
        "M",
        512,
        512,
        512,
        "M",
        512,
        512,
        512,
        "M",
    ],
    "VGG19": [
        64,
        64,
        "M",
        128,
        128,
        "M",
        256,
        256,
        256,
        256,
        "M",
        512,
        512,
        512,
        512,
        "M",
        512,
        512,
        512,
        512,
        "M",
    ],
}


class VGG_net(nn.Module):
    def __init__(self, in_channels=1, num_classes=3):
        super(VGG_net, self).__init__()
        self.in_channels = in_channels
        self.conv_layers = self.create_conv_layers(VGG_types["VGG16"])

        self.fcs = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fcs(x)
        return x

    def create_conv_layers(self, architecture):
        layers = []
        in_channels = self.in_channels

        for x in architecture:
            if type(x) == int:
                out_channels = x

                layers += [
                    nn.Conv2d(
                        in_channels=in_channels,
                        out_channels=out_channels,
                        kernel_size=(3, 3),
                        stride=(1, 1),
                        padding=(1, 1),
                    ),
                    nn.BatchNorm2d(x),
                    nn.ReLU(),
                ]
                in_channels = x
            elif x == "M":
                layers += [nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))]

        return nn.Sequential(*layers)

In [ ]:
model = VGG_net()
print(model)


In [ ]:
model.load_state_dict(torch.load(r"C:\Users\HP\Desktop\Model\VGG16\VGG16_Without_Aug\VGG16_.pth", map_location={'cuda:0': 'cpu'}))

model.eval()

In [ ]:
from PIL import Image

In [ ]:
def make_prediciton(image_path):
    image = Image.open(image_path).convert("L")
    pretrained_size = 224
    pretrained_means = [0.5]
    pretrained_stds = [0.5]

    train_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.5),(0.5)),
            ])
        
    image = train_transforms(image)    
    image = image.reshape(1, 1, 224, 224)
    predict = model(image)
    softmax = nn.Softmax(dim=1)
    predict = softmax(predict)
    probability , prediction = torch.max(predict, dim = 1)
    #converting torch tensor into numpy array
    probability = probability.detach().numpy()[0]
    prediction = prediction.detach().numpy()[0]
    
    return probability, prediction

In [ ]:
# make_prediciton(test_dataset[0][0])

In [ ]:
make_prediciton(r"C:\Users\HP\Desktop\Model\DATASET\Testing\malignant\malignant (47).png")

In [ ]:
make_prediciton(r"C:\Users\HP\Desktop\Model\DATASET\Testing\benign\benign (1).png")

In [ ]:
#Wrong Prediction
make_prediciton(r"C:\Users\HP\Desktop\Model\DATASET\Testing\normal\normal (1).png")

In [ ]:
# Hyper parameters
epochs = 30
num_classes = 3
batch_size = 32
learning_rate = 0.001


In [ ]:
# CPU or GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
test_transforms = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
        transforms.Normalize((0.5),(0.5)),
                       ])


valid_transforms = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
        transforms.Normalize((0.5),(0.5)),
                       ])

In [ ]:
test_dataset = torchvision.datasets.ImageFolder(root= r"C:\Users\HP\Desktop\Model\DATASET\Testing", transform = test_transforms)
# test_dataset = torchvision.datasets.ImageFolder(root= r"C:\Users\HP\Desktop\Model\DATASET", transform = test_transforms)

In [ ]:
test_dataset

In [ ]:
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle=True, num_workers=2)

In [ ]:
from tqdm import tqdm

In [ ]:
test_dataset[100]

In [ ]:
preds = []
target = []
probabilities = []
individual_prob = []

for images, labels in tqdm(test_loader):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            outputs = nn.Softmax(dim=1)(outputs)
            prob, predicted = torch.max(outputs.data, 1)
            #Tensor from GPU to CPU by converting pytorch numpy to tensor array
            predicted = predicted.cpu().numpy()
            outputs = outputs.cpu().detach().numpy()
            prob = prob.cpu().detach().numpy()
            labels = labels.cpu().numpy()
            preds.append(predicted)
            target.append(labels)
            #Finding probabilies for each probabilities
            probabilities.append(list(outputs))
            individual_prob.append(list(prob))
            
        

Converting 2-D into 1-D array

In [ ]:
probabilities1 = []
individual_prob1 = []
preds1 = []
target1 = []

for i in probabilities:
  for j in i:
    probabilities1.append(j)

for i in individual_prob:
  for j in i:
    individual_prob1.append(j)
    
for i in preds:
  for j in i:
    preds1.append(j)
    
for i in target:
  for j in i:
    target1.append(j)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
accuracy_score(target1,preds1)

In [ ]:
print(classification_report(target1,preds1))

In [ ]:
confusion_matrix(target1,preds1)

In [ ]:
import matplotlib.pyplot as plt
import numpy
from sklearn .metrics import roc_auc_score, roc_curve
import seaborn as sns


In [ ]:
test_dataset.classes

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def plot_cm(confusion):
    plt.figure(figsize=(8, 6))
    sns.set(font_scale=1.2)
    sns.heatmap(confusion, annot=True, fmt="d", cmap="Greens", square=True, 
                cbar=False, annot_kws={"size": 12}, linewidths=0.5)
    plt.xlabel('Predicted labels', fontsize=14)
    plt.ylabel('True labels', fontsize=14)
    plt.title('Confusion Matrix', fontsize=16)
    classes = ['Benign', 'Malignant', 'Normal'] 
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks + 0.5, classes, fontsize=12)
    plt.yticks(tick_marks + 0.5, classes, fontsize=12)
    plt.tight_layout()
    plt.show()

from sklearn.metrics import confusion_matrix

# Assuming target1 and preds1 are the true labels and predicted labels, respectively
confusion = confusion_matrix(target1, preds1, labels=[0, 1, 2])
plot_cm(confusion)


In [ ]:
auc = np.round(roc_auc_score(target1, probabilities1, multi_class = 'ovr'), 3)
print("Auc score for our sample data is {}". format(auc))

In [ ]:
probabilities2 = np.array([i for i in probabilities1])

In [ ]:
def plot_multiclass_roc(y_score, y_test, n_classes, figsize=(17, 6)):

    # structures
    fpr = dict()
    tpr = dict()
    threshold = dict()

    for i in range(n_classes):    
      fpr[i], tpr[i], threshold[i] = roc_curve(target1, probabilities2[:,i], pos_label=i)
    # roc for each class
    fig, ax = plt.subplots(figsize=figsize)
    ax.plot([0, 1], [0, 1], 'k--')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title('Receiver Operating Characteristic ')
    for i in range(n_classes):
        ax.plot(fpr[i], tpr[i], label = 'class '+ test_dataset.classes[i])
    ax.legend(loc="best")
    ax.grid(alpha=.4)
    sns.despine()
    plt.show()

plot_multiclass_roc(probabilities2, target1, n_classes=3, figsize=(12,8))